In [1]:
# Análisis y Modelo de Regresión Logística para Predecir la Calidad del Vino
# =========================================================================

# ## Introducción
# Este notebook está diseñado para cargar, explorar y preprocesar el dataset de calidad de vinos, y entrenar un modelo de regresión logística utilizando PySpark en un entorno de Databricks.

# ## 1. Configuración de Databricks y Bibliotecas
# Importamos las bibliotecas necesarias y configuramos la conexión con Databricks.

import sys
sys.executable

import findspark
findspark.init()

import pandas as pd
import pyspark

import pandas as pd
import numpy as np
#import pyspark.pandas as ps
from pyspark.sql import SparkSession
import matplotlib as plt
import seaborn as sns
from pyspark.sql.functions import col, sum



from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator





In [2]:

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('WineQualityPrediction')\
        .getOrCreate()

In [3]:
# ## 2. Carga de Datos
# Cargamos el archivo CSV con los datos del vino y revisamos las primeras filas.

# Cargar el archivo CSV

gt = spark.read.csv('data/winequality-red.csv', 
                       inferSchema = True,
                       sep=";", 
                       header = True)


In [4]:
print(type(gt))

<class 'pyspark.sql.dataframe.DataFrame'>


In [5]:
df = gt

In [6]:
# Convertir la columna "quality" a una variable binaria en una columna llamada "label"

df = df.withColumn("label", (col("quality") >= 6).cast("int"))


In [7]:
# Selección de características

feature_columns = [c for c in df.columns if c not in ["quality", "label"]]


In [8]:
# Ensamblaje de las características en un solo vector

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df)

In [9]:
# Estandarización de las características

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)

In [10]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [11]:
# Entrenar el modelo de regresión logística
lr = LogisticRegression(featuresCol="scaledFeatures", labelCol="label")
lr_model = lr.fit(train_data)

In [12]:
# ## 6. Evaluación del Modelo

# Realizar predicciones en el conjunto de prueba
predictions = lr_model.transform(test_data)

In [13]:
# Evaluador para medir el área bajo la curva ROC
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predictions)
print(f"Área bajo la curva ROC: {roc_auc:.2f}")

Área bajo la curva ROC: 0.84


In [14]:
# Calcular la precisión y mostrar el informe de clasificación
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(predictions)
print(f"Exactitud del modelo: {accuracy:.2f}")

Exactitud del modelo: 0.77


In [15]:
# Mostrar una muestra de las predicciones
predictions.select("label", "prediction", "probability").show(5)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|    1|       1.0|[0.01812103566953...|
|    1|       1.0|[0.25500976345328...|
|    0|       0.0|[0.79508949677236...|
|    0|       0.0|[0.76399102429231...|
|    1|       1.0|[0.04814519001736...|
+-----+----------+--------------------+
only showing top 5 rows

